<a href="https://colab.research.google.com/github/mattambrogi/claude-prefill-and-stop-sequence/blob/main/words_in_claudes_mouth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilizing Claude's Prefill ability for more reliable formatting

In [ ]:
!pip install anthropic

In [ ]:
import os
os.environ["ANTHROPIC_API_KEY"] = ""

In [ ]:
import anthropic

client = anthropic.Anthropic()

In [ ]:
import json

In [ ]:
def pretty_print_json(message):
  response_text = message.content[0].text
  recovered_json_response = f'[{response_text}'

  # Parse the JSON string
  try:
      parsed_json = json.loads(recovered_json_response)

      # Print the formatted JSON
      return json.dumps(parsed_json, indent=2)
  except json.JSONDecodeError as e:
      error_message = f"Error parsing JSON: {e} Original content: {response_text}"
      return error_message

**Prompt**

I have an API endpoint that returns Todos for a Todo application. Here is an example of the response:

<example>
[
  {
    "userId": 2,
    "id": 12,
    "title": "in quibusdam tempore odit est dolorem",
    "body": "itaque id aut magnam\npraesentium quia et ea odit et ea voluptas et\nsapiente quia nihil amet occaecati quia id voluptatem\nincidunt ea est distinctio odio"
  },
  {
    "userId": 2,
    "id": 13,
    "title": "dolorum ut in voluptas mollitia et saepe quo animi",
    "body": "aut dicta possimus sint mollitia voluptas commodi quo doloremque\niste corrupti reiciendis voluptatem eius rerum\nsit cumque quod eligendi laborum minima\nperferendis recusandae assumenda consectetur porro architecto ipsum ipsam"
  }
]
</example>

Can you help me generate five more examples? Here are more specific instructions:

<instructions>
* Generate 5 todos following the schema above
* Vary the userId between 1 and 5 randomly
* Please make the titles and body's real sentences that might be on someone who has chores around the house's todo list for the day
</instructions>

## Without pre-fill

In [ ]:
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.1,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "I have an API endpoint that returns Todos for a Todo application. Here is an example of the response:\n\n<example>\n[\n  {\n    \"userId\": 2,\n    \"id\": 12,\n    \"title\": \"in quibusdam tempore odit est dolorem\",\n    \"body\": \"itaque id aut magnam\\npraesentium quia et ea odit et ea voluptas et\\nsapiente quia nihil amet occaecati quia id voluptatem\\nincidunt ea est distinctio odio\"\n  },\n  {\n    \"userId\": 2,\n    \"id\": 13,\n    \"title\": \"dolorum ut in voluptas mollitia et saepe quo animi\",\n    \"body\": \"aut dicta possimus sint mollitia voluptas commodi quo doloremque\\niste corrupti reiciendis voluptatem eius rerum\\nsit cumque quod eligendi laborum minima\\nperferendis recusandae assumenda consectetur porro architecto ipsum ipsam\"\n  }\n]\n</example>\n\nCan you help me generate five more examples? Here are more specific instructions:\n\n<instructions>\n* Generate 5 todos following the schema above\n* Vary the userId between 1 and 5 randomly\n* Please make the titles and body's real sentences that might be on someone who has chores around the house's todo list for the day \n</instructions>"
                }
            ]
        }
    ]
)
print(message.content)

[TextBlock(text='Certainly! Here are 5 more examples of todos following the schema and instructions you provided:\n\n[\n  {\n    "userId": 3,\n    "id": 14,\n    "title": "Clean the kitchen and organize pantry",\n    "body": "Wipe down all countertops and appliances. Sweep and mop the floor. Sort through pantry items, discard expired products, and reorganize shelves for better accessibility. Make a list of groceries needed for restocking."\n  },\n  {\n    "userId": 1,\n    "id": 15,\n    "title": "Mow the lawn and trim hedges",\n    "body": "Start by mowing the front and back lawns. Adjust mower height for optimal grass health. Use hedge trimmers to shape bushes and maintain a neat appearance. Rake and dispose of grass clippings and hedge trimmings."\n  },\n  {\n    "userId": 5,\n    "id": 16,\n    "title": "Do laundry and iron clothes for the week",\n    "body": "Sort clothes by color and fabric type. Wash and dry all loads. Fold and put away casual clothes. Iron work shirts and pants

**We don't want the "Certainly!...**

In [ ]:
print(pretty_print_json(message))

Error parsing JSON: Expecting value: line 1 column 2 (char 1) Original content: Certainly! Here are 5 more examples of todos following the schema and instructions you provided:

[
  {
    "userId": 3,
    "id": 14,
    "title": "Clean the kitchen and organize pantry",
    "body": "Wipe down all countertops and appliances. Sweep and mop the floor. Sort through pantry items, discard expired products, and reorganize shelves for better accessibility. Make a list of groceries needed for restocking."
  },
  {
    "userId": 1,
    "id": 15,
    "title": "Mow the lawn and trim hedges",
    "body": "Start by mowing the front and back lawns. Adjust mower height for optimal grass health. Use hedge trimmers to shape bushes and maintain a neat appearance. Rake and dispose of grass clippings and hedge trimmings."
  },
  {
    "userId": 5,
    "id": 16,
    "title": "Do laundry and iron clothes for the week",
    "body": "Sort clothes by color and fabric type. Wash and dry all loads. Fold and put awa

## With pre-fill

In [ ]:
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.1,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "I have an API endpoint that returns Todos for a Todo application. Here is an example of the response:\n\n<example>\n[\n  {\n    \"userId\": 2,\n    \"id\": 12,\n    \"title\": \"in quibusdam tempore odit est dolorem\",\n    \"body\": \"itaque id aut magnam\\npraesentium quia et ea odit et ea voluptas et\\nsapiente quia nihil amet occaecati quia id voluptatem\\nincidunt ea est distinctio odio\"\n  },\n  {\n    \"userId\": 2,\n    \"id\": 13,\n    \"title\": \"dolorum ut in voluptas mollitia et saepe quo animi\",\n    \"body\": \"aut dicta possimus sint mollitia voluptas commodi quo doloremque\\niste corrupti reiciendis voluptatem eius rerum\\nsit cumque quod eligendi laborum minima\\nperferendis recusandae assumenda consectetur porro architecto ipsum ipsam\"\n  }\n]\n</example>\n\nCan you help me generate five more examples? Here are more specific instructions:\n\n<instructions>\n* Generate 5 todos following the schema above\n* Vary the userId between 1 and 5 randomly\n* Please make the titles and body's real sentences that might be on someone who has chores around the house's todo list for the day \n</instructions>"
                }
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "["
                }
            ]
        }
    ]
)

**No more "Certainly!...**

In [ ]:
print(pretty_print_json(message))

[
  {
    "userId": 3,
    "id": 14,
    "title": "Clean the kitchen and organize pantry",
    "body": "Wipe down all countertops and appliances. Sweep and mop the floor. Sort through pantry items, discarding expired products. Organize remaining items by category for easy access."
  },
  {
    "userId": 1,
    "id": 15,
    "title": "Mow the lawn and trim hedges",
    "body": "Start by mowing the front lawn, then move to the backyard. Ensure to empty grass clippings into compost bin. After mowing, trim the hedges along the fence line. Clean up any debris and put away gardening tools."
  },
  {
    "userId": 5,
    "id": 16,
    "title": "Do laundry and fold clothes",
    "body": "Separate clothes into whites, darks, and colors. Start with the largest load first. While waiting, gather all clean clothes from previous loads. Fold and put away all clean laundry, including towels and bedsheets."
  },
  {
    "userId": 2,
    "id": 17,
    "title": "Prepare weekly meal plan and grocery list"

## With pre-fill and stop sequence

**Let's make sure there's no extra content at the end either**

In [ ]:
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.1,
    stop_sequences=["]"],
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "I have an API endpoint that returns Todos for a Todo application. Here is an example of the response:\n\n<example>\n[\n  {\n    \"userId\": 2,\n    \"id\": 12,\n    \"title\": \"in quibusdam tempore odit est dolorem\",\n    \"body\": \"itaque id aut magnam\\npraesentium quia et ea odit et ea voluptas et\\nsapiente quia nihil amet occaecati quia id voluptatem\\nincidunt ea est distinctio odio\"\n  },\n  {\n    \"userId\": 2,\n    \"id\": 13,\n    \"title\": \"dolorum ut in voluptas mollitia et saepe quo animi\",\n    \"body\": \"aut dicta possimus sint mollitia voluptas commodi quo doloremque\\niste corrupti reiciendis voluptatem eius rerum\\nsit cumque quod eligendi laborum minima\\nperferendis recusandae assumenda consectetur porro architecto ipsum ipsam\"\n  }\n]\n</example>\n\nCan you help me generate five more examples? Here are more specific instructions:\n\n<instructions>\n* Generate 5 todos following the schema above\n* Vary the userId between 1 and 5 randomly\n* Please make the titles and body's real sentences that might be on someone who has chores around the house's todo list for the day \n</instructions>"
                }
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "["
                }
            ]
        }
    ]
)

In [ ]:
def pretty_print_json_stop_seq(message):
  response_text = message.content[0].text
  recovered_json_response = f'[{response_text}]'

  # Parse the JSON string
  try:
      parsed_json = json.loads(recovered_json_response)

      # Print the formatted JSON
      return json.dumps(parsed_json, indent=2)
  except json.JSONDecodeError as e:
      error_message = f"Error parsing JSON: {e} Original content: {response_text}"
      return error_message

In [ ]:
print(pretty_print_json_stop_seq(message))

[
  {
    "userId": 3,
    "id": 14,
    "title": "Clean the kitchen and organize pantry",
    "body": "Wipe down all countertops and appliances. Sweep and mop the floor. Sort through pantry items, discarding expired products. Organize remaining items by category for easy access."
  },
  {
    "userId": 1,
    "id": 15,
    "title": "Mow the lawn and trim hedges",
    "body": "Start by mowing the front lawn, then move to the backyard. Ensure to empty grass clippings into compost bin. After mowing, trim the hedges along the fence line. Clean up any debris and put away gardening tools."
  },
  {
    "userId": 5,
    "id": 16,
    "title": "Do laundry and fold clothes",
    "body": "Separate clothes into whites, darks, and colors. Run each load through washer and dryer. Once dry, fold clothes neatly and put away in respective drawers and closets. Don't forget to wash and change bed sheets."
  },
  {
    "userId": 2,
    "id": 17,
    "title": "Grocery shopping for the week",
    "body": "